# 1. Подготовка данных

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import xgboost as xgb
import optuna
import mlflow

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.utils import compute_sample_weight
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline

mlflow.set_tracking_uri("http://127.0.0.1:8000")
client=mlflow.MlflowClient(tracking_uri='http://127.0.0.1:8000')

In [2]:
df = pd.read_csv('Churn.csv')

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
# проверим наличие явных дубликатов:
df.duplicated().sum()

0

In [6]:
# посмотрим уникальные значения в столбце Geography:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
# посмотрим уникальные значения в столбце Tenure:
df['Tenure'].value_counts().sort_values(ascending=False)

Tenure
1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: count, dtype: int64

In [8]:
# посмотрим уникальные значения в столбце NumOfProducts:
df['NumOfProducts'].value_counts()

NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

In [9]:
# посмотрим уникальные значения в столбце HasCrCard:
df['HasCrCard'].value_counts()

HasCrCard
1    7055
0    2945
Name: count, dtype: int64

In [10]:
# посмотрим уникальные значения в столбце IsActiveMember:
df['IsActiveMember'].value_counts()

IsActiveMember
1    5151
0    4849
Name: count, dtype: int64

In [11]:
# посчитаем классы в целевом признаке:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

Можно отметить, что дубликаты в датасете отсутствуют, в столбце Tenure есть около тысячи пропусков, а также несколько столбцов лучше вообще удалить, из них:

- RowNumber - это индекс строки в данных, важной информации не содержит;
- CustomerId - уникальный идентификатор клиента, по нему мы проверили дубликаты и теперь его можно удалить;
- Surname - фамилия, которая тоже не имеет связи с оттоком клиентов.

После проверки датасета на сбалансированность классов в таргете мы видим, что объектов отрицательного класса намного больше, но все равно уходит примерно 20% клиентов. В столбце Geography находятся всего 3 уникальных значения: France, Germany и Spain. В столбце Tenure (сколько лет человек является клиентом банка) есть 909 пропусков. Заполним их значением -1. Есть несколько бинарных признаков (HasCrCard, IsActiveMember), которые затем выделим в отдельную категорию признаков.

In [12]:
# удалим ненужные столбцы:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [13]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [15]:
# сохраним признаки и целевой признак в отдельных переменных:
y = df['Exited']
X = df.drop('Exited', axis=1)

In [16]:
# разобьем исходные данные на обучающую и тестовую выборки в соотношении 4:1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)

In [17]:
# посмотрим на размеры выборок:
print('Размер обучающей выборки:', X_train.shape)
print('Размер тестовой выборки:', X_test.shape)

Размер обучающей выборки: (8000, 10)
Размер тестовой выборки: (2000, 10)


Для автоматизации процесса подготовим численный и категориальный пайплайны.
- Попробуем разные стратегии заполнения пропусков для SimpleImputer.
- Объединим пайплайны классом ColumnTransformer.
-
-

In [18]:
# внесем в переменные списки все численные и категориальные признаки:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(exclude=['object']).columns.tolist()
binary_features = ['HasCrCard', 'IsActiveMember']
numeric_features = list(set(numeric_features) - set(binary_features))

In [19]:
numeric_features

['Tenure', 'NumOfProducts', 'EstimatedSalary', 'Balance', 'CreditScore', 'Age']

In [20]:
categorical_features

['Geography', 'Gender']

In [21]:
binary_features

['HasCrCard', 'IsActiveMember']

# 2. Обучение модели без учёта дисбаланса классов (XGBoost)

In [22]:
pipe_cat = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

pipe_num = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()) 
])

preprocessor = ColumnTransformer([
    ("cat", pipe_cat, categorical_features),
    ("num", pipe_num, numeric_features)

], remainder='passthrough')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(random_state=12345))
])

In [23]:
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('enc',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Geography', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value=-1)),
                                                                  ('pt',
                                                                   PowerTransformer())]),
                                                  ['Balance', 'NumOfProducts',
                                                   'Age', 'EstimatedSalary',
                                                   'Tenure',
                                                   'CreditSco...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=12345, ...))])

In [24]:
def optuna_optimize(trial):
    param_grid = {
        'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 250),
        'model__max_depth': trial.suggest_int('model__max_depth', 2, 10),
        'model__eta': trial.suggest_float('model__eta', 0.01, 1.5)
    }

    pipe.set_params(**param_grid)
    score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [25]:
study_xgb = optuna.create_study(study_name="XGBoost Optuna Optimization Imbalanced",
                                direction='maximize')

[I 2023-11-21 19:08:31,963] A new study created in memory with name: XGBoost Optuna Optimization Imbalanced


In [26]:
study_xgb.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 19:08:36,569] Trial 0 finished with value: 0.5328481157436518 and parameters: {'model__n_estimators': 212, 'model__max_depth': 8, 'model__eta': 1.139093211207534}. Best is trial 0 with value: 0.5328481157436518.
[I 2023-11-21 19:08:37,920] Trial 1 finished with value: 0.5830631477453424 and parameters: {'model__n_estimators': 187, 'model__max_depth': 3, 'model__eta': 0.1889150219662568}. Best is trial 1 with value: 0.5830631477453424.
[I 2023-11-21 19:08:39,469] Trial 2 finished with value: 0.5614947767310493 and parameters: {'model__n_estimators': 187, 'model__max_depth': 4, 'model__eta': 0.2730866247457957}. Best is trial 1 with value: 0.5830631477453424.
[I 2023-11-21 19:08:40,782] Trial 3 finished with value: 0.5570136427243216 and parameters: {'model__n_estimators': 214, 'model__max_depth': 2, 'model__eta': 1.290343631563269}. Best is trial 1 with value: 0.5830631477453424.
[I 2023-11-21 19:08:42,587] Trial 4 finished with value: 0.5153097620111223 and parameters: {'

In [27]:
print("Наилучшие значения гиперпараметров: {}".format(study_xgb.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb.best_value))

Наилучшие значения гиперпараметров: {'model__n_estimators': 166, 'model__max_depth': 4, 'model__eta': 0.12872229943826174}
f1_score на обучающем наборе: 0.588


In [28]:
with mlflow.start_run(nested=True):

    best_params = study_xgb.best_params
    best_score = study_xgb.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    pipe.set_params(**best_params)
    pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(pipe, 'xgb_pipeline_imbalanced')

# 3. Обучение моделей с учётом дисбаланса классов
## 3.1 Назначение разных весов классам (cost-sensitive learning)
### 3.1.1 Модель XGBoost

In [29]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)

], remainder='passthrough')

# pipe = Pipeline([
#     ('preprocessor', preprocessor),
#     ('model', xgb.XGBClassifier(random_state=12345, scale_pos_weight=3.91)) # где это написать??? здесь?
# ])

In [30]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param, 
                                  scale_pos_weight=3.91,
                                  random_state=12345) # или здесь??
    
    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [31]:
study_xgb_cw = optuna.create_study(study_name="XGBoost Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-21 19:29:28,127] A new study created in memory with name: XGBoost Optuna Optimization Class Weight


In [32]:
study_xgb_cw.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 19:29:32,930] Trial 0 finished with value: 0.5747088943116656 and parameters: {'n_estimators': 179, 'max_depth': 5, 'eta': 0.31526074068540255}. Best is trial 0 with value: 0.5747088943116656.
[I 2023-11-21 19:29:35,133] Trial 1 finished with value: 0.5385297240311625 and parameters: {'n_estimators': 202, 'max_depth': 8, 'eta': 1.361722569838879}. Best is trial 0 with value: 0.5747088943116656.
[I 2023-11-21 19:29:36,331] Trial 2 finished with value: 0.5783629221099658 and parameters: {'n_estimators': 215, 'max_depth': 2, 'eta': 0.7625676660072352}. Best is trial 2 with value: 0.5783629221099658.
[I 2023-11-21 19:29:37,365] Trial 3 finished with value: 0.5328472432122172 and parameters: {'n_estimators': 57, 'max_depth': 4, 'eta': 1.3740757281548324}. Best is trial 2 with value: 0.5783629221099658.
[I 2023-11-21 19:29:40,501] Trial 4 finished with value: 0.5800911580480638 and parameters: {'n_estimators': 207, 'max_depth': 9, 'eta': 0.17032075418647902}. Best is trial 4 wi

In [33]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_cw.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_cw.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 140, 'max_depth': 2, 'eta': 0.15787609916936599}
f1_score на обучающем наборе: 0.610


In [35]:
with mlflow.start_run(nested=True):

    best_params = study_xgb_cw.best_params
    best_score = study_xgb_cw.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBClassifier(**best_params,
                                    scale_pos_weight=3.91,
                                    random_state=12345))
    ])

    # xgb_pipe.set_params(**best_params)
    xgb_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(xgb_pipe, 'xgb_pipeline_class_weight')

### 3.1.2 Случайный лес

In [36]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param, 
                                       class_weight='balanced',
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [37]:
study_rf = optuna.create_study(study_name="RandomForest Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-21 21:06:58,621] A new study created in memory with name: RandomForest Optuna Optimization Class Weight


In [38]:
study_rf.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 21:07:32,249] Trial 0 finished with value: 0.5405686599771671 and parameters: {'n_estimators': 233, 'max_depth': 2, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.5405686599771671.
[I 2023-11-21 21:07:43,626] Trial 1 finished with value: 0.576631818004367 and parameters: {'n_estimators': 203, 'max_depth': 4, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.576631818004367.
[I 2023-11-21 21:08:00,336] Trial 2 finished with value: 0.6084544815780663 and parameters: {'n_estimators': 172, 'max_depth': 7, 'min_samples_leaf': 6}. Best is trial 2 with value: 0.6084544815780663.
[I 2023-11-21 21:08:06,928] Trial 3 finished with value: 0.6103458175368759 and parameters: {'n_estimators': 55, 'max_depth': 8, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.6103458175368759.
[I 2023-11-21 21:08:13,809] Trial 4 finished with value: 0.6140754881002055 and parameters: {'n_estimators': 51, 'max_depth': 10, 'min_samples_leaf': 2}. Best is trial 4 with value: 0.61407548810

In [39]:
print("Наилучшие значения гиперпараметров Random Forest: {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf.best_value))

Наилучшие значения гиперпараметров Random Forest: {'n_estimators': 276, 'max_depth': 10, 'min_samples_leaf': 4}
f1_score на обучающем наборе: 0.618


In [40]:
with mlflow.start_run(nested=True):

    best_params = study_rf.best_params
    best_score = study_rf.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(**best_params,
                                         class_weight='balanced',
                                         random_state=12345))
    ])

    rf_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(rf_pipe, 'rf_pipeline_class_weight')

### 3.1.3 Логистическая регрессия

In [41]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')


In [42]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }
    
    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) 
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 
        
    estimator = LogisticRegression(**param,  
                                   class_weight='balanced',
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [43]:
# optuna.logging.set_verbosity(optuna.logging.WARNING) # i do not want to see trail information
study_log_reg = optuna.create_study(study_name="LogisticRegression Optuna Optimization Class Weight",
                                direction='maximize')

[I 2023-11-21 21:43:38,469] A new study created in memory with name: LogisticRegression Optuna Optimization Class Weight


In [44]:
study_log_reg.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 21:43:39,172] Trial 0 finished with value: 0.49096987218775245 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.49096987218775245.
[I 2023-11-21 21:43:40,451] Trial 1 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 33.25204216640461, 'l1_ratio': 0.417298995590638}. Best is trial 1 with value: 0.4910843408008952.
[I 2023-11-21 21:43:41,569] Trial 2 finished with value: 0.4910843408008952 and parameters: {'solver': 'saga', 'C': 60.712685504378676, 'l1_ratio': 0.6687692357927916}. Best is trial 1 with value: 0.4910843408008952.
[I 2023-11-21 21:43:42,265] Trial 3 finished with value: 0.49096987218775245 and parameters: {'solver': 'lbfgs'}. Best is trial 1 with value: 0.4910843408008952.
[I 2023-11-21 21:43:42,879] Trial 4 finished with value: 0.49096987218775245 and parameters: {'solver': 'lbfgs'}. Best is trial 1 with value: 0.4910843408008952.
[I 2023-11-21 21:43:43,565] Trial 5 finished with value: 0.4910843408008952 and p

In [45]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'saga', 'C': 33.25204216640461, 'l1_ratio': 0.417298995590638}
f1_score на обучающем наборе: 0.491


In [46]:
with mlflow.start_run(nested=True):

    best_params = study_log_reg.best_params
    best_score = study_log_reg.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(**best_params,
                                     class_weight='balanced',
                                     random_state=12345))
    ])

    log_reg_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(log_reg_pipe, 'log_reg_pipeline_class_weight')

### 3.1.4 Метод опорных векторов

In [47]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.001, 200)  # какие значения может принимать С 
    }
 
    estimator = SVC(**param, 
                    class_weight='balanced',
                    random_state=12345,
                    probability=True)
    
    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [48]:
study_svc = optuna.create_study(study_name="SVC Optuna Optimization Class Weight",
                                    direction='maximize')

[I 2023-11-21 21:44:49,507] A new study created in memory with name: SVC Optuna Optimization Class Weight


In [49]:
study_svc.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 21:45:04,455] Trial 0 finished with value: 0.5476048478545517 and parameters: {'C': 32.544553574348555}. Best is trial 0 with value: 0.5476048478545517.
[I 2023-11-21 21:45:34,312] Trial 1 finished with value: 0.5123467551999545 and parameters: {'C': 170.55453431442686}. Best is trial 0 with value: 0.5476048478545517.
[I 2023-11-21 21:45:45,663] Trial 2 finished with value: 0.5617629950459692 and parameters: {'C': 13.856562760862762}. Best is trial 2 with value: 0.5617629950459692.
[I 2023-11-21 21:45:59,499] Trial 3 finished with value: 0.5490667905531239 and parameters: {'C': 30.01021095175272}. Best is trial 2 with value: 0.5617629950459692.
[I 2023-11-21 21:46:12,472] Trial 4 finished with value: 0.5538329685935311 and parameters: {'C': 24.881568001412848}. Best is trial 2 with value: 0.5617629950459692.
[I 2023-11-21 21:46:27,482] Trial 5 finished with value: 0.5445946361107652 and parameters: {'C': 40.05035755059586}. Best is trial 2 with value: 0.5617629950459692.


In [50]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 2.7064620319531265}
f1_score на обучающем наборе: 0.579


In [51]:
with mlflow.start_run(nested=True):

    best_params = study_svc.best_params
    best_score = study_svc.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', SVC(**best_params,
                      class_weight='balanced',
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(svc_pipe, 'svc_pipeline_class_weight')

## 3.2 Генерация синтетических данных (SMOTE)
### 3.2.1 Модель XGBoost

In [23]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [53]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param,
                                  random_state=12345)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [54]:
study_xgb_smote = optuna.create_study(study_name="XGBoost Optuna Optimization SMOTE",
                                direction='maximize')

[I 2023-11-21 22:06:26,970] A new study created in memory with name: XGBoost Optuna Optimization SMOTE


In [55]:
study_xgb_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 22:06:30,207] Trial 0 finished with value: 0.5327918569184967 and parameters: {'n_estimators': 119, 'max_depth': 6, 'eta': 1.1916220912659359}. Best is trial 0 with value: 0.5327918569184967.
[I 2023-11-21 22:06:32,805] Trial 1 finished with value: 0.5536180843793439 and parameters: {'n_estimators': 82, 'max_depth': 7, 'eta': 0.9402405475635459}. Best is trial 1 with value: 0.5536180843793439.
[I 2023-11-21 22:06:37,107] Trial 2 finished with value: 0.5489485401007134 and parameters: {'n_estimators': 186, 'max_depth': 7, 'eta': 0.9312755084847804}. Best is trial 1 with value: 0.5536180843793439.
[I 2023-11-21 22:06:40,774] Trial 3 finished with value: 0.542279472487669 and parameters: {'n_estimators': 146, 'max_depth': 7, 'eta': 1.2236922522352667}. Best is trial 1 with value: 0.5536180843793439.
[I 2023-11-21 22:06:44,208] Trial 4 finished with value: 0.548261155183717 and parameters: {'n_estimators': 108, 'max_depth': 8, 'eta': 1.2383951591882436}. Best is trial 1 with 

In [56]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_smote.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 131, 'max_depth': 3, 'eta': 0.11250589108448067}
f1_score на обучающем наборе: 0.626


In [57]:
with mlflow.start_run(nested=True):

    best_params = study_xgb_smote.best_params
    best_score = study_xgb_smote.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', xgb.XGBClassifier(**best_params,
                                    random_state=12345))
    ])

    xgb_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(xgb_pipe, 'xgb_pipeline_smote')

### 3.2.2 Случайный лес

In [58]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param,
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [59]:
study_rf_smote = optuna.create_study(study_name="RandomForest Optuna Optimization SMOTE",
                               direction='maximize')

[I 2023-11-21 22:09:50,994] A new study created in memory with name: RandomForest Optuna Optimization SMOTE


In [60]:
study_rf_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 22:09:54,257] Trial 0 finished with value: 0.5845789288318117 and parameters: {'n_estimators': 53, 'max_depth': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.5845789288318117.
[I 2023-11-21 22:10:17,186] Trial 1 finished with value: 0.6040930649033174 and parameters: {'n_estimators': 292, 'max_depth': 7, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.6040930649033174.
[I 2023-11-21 22:10:23,399] Trial 2 finished with value: 0.5451778740423928 and parameters: {'n_estimators': 183, 'max_depth': 2, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.6040930649033174.
[I 2023-11-21 22:10:28,024] Trial 3 finished with value: 0.5987112504821926 and parameters: {'n_estimators': 60, 'max_depth': 6, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.6040930649033174.
[I 2023-11-21 22:10:44,331] Trial 4 finished with value: 0.5994660549226251 and parameters: {'n_estimators': 229, 'max_depth': 6, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.6040930649

In [61]:
print("Наилучшие значения гиперпараметров RandomForest: {}".format(study_rf_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf_smote.best_value))

Наилучшие значения гиперпараметров RandomForest: {'n_estimators': 213, 'max_depth': 9, 'min_samples_leaf': 5}
f1_score на обучающем наборе: 0.615


In [62]:
with mlflow.start_run(nested=True):

    best_params = study_rf_smote.best_params
    best_score = study_rf_smote.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', RandomForestClassifier(**best_params,
                                         random_state=12345))
    ])

    rf_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(rf_pipe, 'rf_pipeline_smote')

### 3.2.3 Логистическая регрессия

In [63]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [64]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }

    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) # какие тут значения?
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 

    estimator = LogisticRegression(**param,
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [65]:
study_log_reg_smote = optuna.create_study(study_name="LogisticRegression Optuna Optimization SMOTE",
                                    direction='maximize')

[I 2023-11-21 22:39:31,648] A new study created in memory with name: LogisticRegression Optuna Optimization SMOTE


In [66]:
study_log_reg_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 22:39:32,403] Trial 0 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-21 22:39:33,100] Trial 1 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-21 22:39:33,985] Trial 2 finished with value: 0.49305864444463354 and parameters: {'solver': 'saga', 'C': 49.131383767424836, 'l1_ratio': 0.57137673581926}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-21 22:39:34,860] Trial 3 finished with value: 0.49305864444463354 and parameters: {'solver': 'saga', 'C': 98.63896983912585, 'l1_ratio': 0.2899681312441582}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-21 22:39:35,574] Trial 4 finished with value: 0.4931691414132686 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.4931691414132686.
[I 2023-11-21 22:39:36,481] Trial 5 finished with value: 0.49305864444463354 and par

In [67]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg_smote.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'saga', 'C': 0.10059694466672653, 'l1_ratio': 0.9962611634422451}
f1_score на обучающем наборе: 0.493


In [68]:
with mlflow.start_run(nested=True):

    best_params = study_log_reg_smote.best_params
    best_score = study_log_reg_smote.best_value
    mlflow.log_params(best_params)
    mlflow.log_metric('f1', best_score)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', LogisticRegression(**best_params,
                                     random_state=12345))
    ])

    log_reg_pipe.fit(X_train, y_train)

    mlflow.sklearn.log_model(log_reg_pipe, 'log_reg_pipeline_smote')

### 3.2.4 Метод опорных векторов

In [69]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.001, 200)  # какие значения может принимать С 
    }

    estimator = SVC(**param,
                    random_state=12345,
                    probability=True)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    
    return score

In [70]:
study_svc_smote = optuna.create_study(study_name="SVC Optuna Optimization SMOTE",
                                direction='maximize')

[I 2023-11-21 22:40:52,138] A new study created in memory with name: SVC Optuna Optimization SMOTE


In [71]:
study_svc_smote.optimize(optuna_optimize, n_trials=100)

[I 2023-11-21 22:41:29,324] Trial 0 finished with value: 0.5252647741225143 and parameters: {'C': 75.21044813489343}. Best is trial 0 with value: 0.5252647741225143.
[I 2023-11-21 22:42:19,757] Trial 1 finished with value: 0.517355823479108 and parameters: {'C': 149.2500094664965}. Best is trial 0 with value: 0.5252647741225143.
[I 2023-11-21 22:42:58,268] Trial 2 finished with value: 0.524153585490968 and parameters: {'C': 79.80086579311777}. Best is trial 0 with value: 0.5252647741225143.
[I 2023-11-21 22:43:36,992] Trial 3 finished with value: 0.5224224855187849 and parameters: {'C': 83.4727088637091}. Best is trial 0 with value: 0.5252647741225143.
[I 2023-11-21 22:44:30,289] Trial 4 finished with value: 0.5141013405011912 and parameters: {'C': 172.95456347128174}. Best is trial 0 with value: 0.5252647741225143.
[I 2023-11-21 22:45:22,637] Trial 5 finished with value: 0.5159608812687078 and parameters: {'C': 161.22874153013962}. Best is trial 0 with value: 0.5252647741225143.
[I 20

In [72]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc_smote.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 1.436363393996341}
f1_score на обучающем наборе: 0.577


In [73]:
with mlflow.start_run(nested=True):

    best_params = study_svc_smote.best_params
    best_score = study_svc_smote.best_value
    mlflow.log_params(best_params)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', SVC(**best_params,
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe.fit(X_train, y_train)
    pred_proba = svc_pipe.predict_proba(X_train)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    mlflow.log_metrics({'f1': best_score,
                        'roc_auc': roc_auc})

    mlflow.sklearn.log_model(svc_pipe, 'svc_pipeline_smote')

In [74]:
colNames = ["XGBoost", "Random Forest", "Logistic Regression", "SVC"]
results = {"Imbalanced":[study_xgb.best_value, '-', '-', '-'],
           "Class Weight":[study_xgb_cw.best_value, 
                           study_rf.best_value, 
                           study_log_reg.best_value, 
                           study_svc.best_value],
           "SMOTE":[study_xgb_smote.best_value, 
                    study_rf_smote.best_value, 
                    study_log_reg_smote.best_value, 
                    study_svc_smote.best_value]}

In [75]:
table = pd.DataFrame.from_dict(results, orient='index', columns=colNames)

In [76]:
table

,XGBoost,Random Forest,Logistic Regression,SVC
Imbalanced,0.588296,-,-,-
Class Weight,0.610398,0.618283,0.491084,0.578716
SMOTE,0.626416,0.614727,0.493398,0.577437


In [27]:
xgb_imbalanced = mlflow.sklearn.load_model('runs:/109ce7a4329544d6bedc74461c63fb94/xgb_pipeline_imbalanced')

In [28]:
pred_proba_xgb_imb = xgb_imbalanced.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_xgb_imb))

ROC-AUC score на тестовой выборке: 0.8700549548007174


In [29]:
xgb_cw = mlflow.sklearn.load_model('runs:/b723baf0ee03493db8ac153265b5caf1/xgb_pipeline_class_weight')

In [31]:
pred_proba_xgb_cw = xgb_cw.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_xgb_cw))

ROC-AUC score на тестовой выборке: 0.867781494900139


In [32]:
rf_cw = mlflow.sklearn.load_model('runs:/bd6c0cd0b02345938e85dfbc193279aa/rf_pipeline_class_weight')

In [33]:
pred_proba_rf_cw = rf_cw.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_rf_cw))

ROC-AUC score на тестовой выборке: 0.8702971075852433


In [34]:
lr_cw = mlflow.sklearn.load_model('runs:/aae3a9d0059a460b85f1bfb51fc7dbd3/log_reg_pipeline_class_weight')

In [35]:
pred_proba_lr_cw = lr_cw.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_lr_cw))

ROC-AUC score на тестовой выборке: 0.7885512631275343


In [42]:
svc_cw = mlflow.sklearn.load_model('runs:/0fdac9a68ae348a388647a10d6507edb/svc_pipeline_class_weight')

In [43]:
pred_proba_svc_cw = svc_cw.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_svc_cw))

AttributeError: predict_proba is not available when probability=False

In [44]:
xgb_smote = mlflow.sklearn.load_model('runs:/0e388b6d46cb442780f28cb35f4e82e1/xgb_pipeline_smote')

In [45]:
pred_proba_xgb_smote = xgb_smote.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_xgb_smote))

ROC-AUC score на тестовой выборке: 0.864531711989339


In [46]:
rf_smote = mlflow.sklearn.load_model('runs:/22ced2eb52fd4c988ec63415190852d2/rf_pipeline_smote')

In [47]:
pred_proba_rf_smote = rf_smote.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_rf_smote))

ROC-AUC score на тестовой выборке: 0.8627039982972187


In [48]:
lr_smote = mlflow.sklearn.load_model('runs:/fe00761f7293499a91d52a349b7b7a29/log_reg_pipeline_smote')

In [49]:
pred_proba_lr_smote = lr_smote.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_lr_smote))

ROC-AUC score на тестовой выборке: 0.7877723640435506


In [50]:
svc_smote = mlflow.sklearn.load_model('runs:/74a6c4d491e14ed291a5ac2e4789d489/svc_pipeline_smote')

In [51]:
pred_proba_svc_smote = svc_smote.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_svc_smote))

AttributeError: predict_proba is not available when probability=False

# 4. Тестирование лучшей модели


In [24]:
best_model = mlflow.sklearn.load_model('runs:/0e388b6d46cb442780f28cb35f4e82e1/xgb_pipeline_smote')

In [25]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('enc',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Geography', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value=-1)),
                                                                  ('pt',
                                                                   PowerTransformer())]),
                                                  ['Balance', 'NumOfProducts',
                                                   'Age', 'EstimatedSalary',
                                                   'Tenure',
                                                   'CreditSco...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=131, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [81]:
# pred_proba = best_model.predict_proba(X_test)[:, 1]
# print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba))

ROC-AUC score на тестовой выборке: 0.864531711989339


In [26]:
pred = best_model.predict(X_test) # почему f1 лучше на тесте, чем на трейне
print('f1 на тестовой выборке:', f1_score(pred, y_test))

f1 на тестовой выборке: 0.6323529411764707
